Prepare for y4q1 (Oct-Dec submission)

First issue. Starting to register EN-TEC biosamples. which have a fairly complex relationships

In [1]:
import pandas
import gcat
import paramiko

In [2]:
from curation_common import *

In [3]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [4]:
spreadsheet_name = 'ENCODE Submission for Y4Q1'

# Register Biosamples

In [5]:
y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
biosample = y4q1.parse('Biosamples', header=0)

created = server.post_sheet('/biosamples/', biosample, verbose=True, dry_run=True)

In [6]:
print(len(created))
#created

0


In [7]:
biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Experiments

In [8]:
y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
experiments = y4q1.parse('Experiments', header=0)

created = server.post_sheet('/experiments/', experiments, verbose=True, dry_run=True)

In [9]:
len(created)

0

In [10]:
experiments.to_excel('/tmp/experiments.xlsx', index=False)

# Register Libraries

In [11]:
y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
libraries = y4q1.parse('Libraries', header=0)

created = server.post_sheet('/libraries/', libraries, verbose=True, dry_run=True)

In [12]:
len(created)

0

In [13]:
libraries.to_excel('/tmp/libraries.xlsx', index=False)

# Register Replicates

In [14]:
y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
replicates = y4q1.parse('Replicates', header=0)

created = server.post_sheet('/replicates/', replicates, verbose=True, dry_run=False)

In [15]:
len(created)

0

In [16]:
replicates.to_excel('/tmp/replicates.xlsx', index=False)

# Create Manifest

In [17]:
def make_manifest(spreadsheet_name, submission_name, submission_root, map_name):
    book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
    libraries = book.parse('Libraries')
    replicates = book.parse('Replicates')
    
    pongo = paramiko.SSHClient()
    pongo.set_missing_host_key_policy(paramiko.WarningPolicy())
    pongo.connect('pongo.cacr.caltech.edu', username='diane')
    sftp = pongo.open_sftp()
    sftp.chdir(submission_root)
    files = 0
    for line in sftp.open(map_name):
        # skip comments
        line = line.strip()
        if len(line) == 0 or line.startswith('#'):
            continue

        library_id, dirname = line.split()
        alias = 'barbara-wold:{}'.format(library_id)
        library_row = libraries[libraries['aliases:array'] == alias]
        replicate_row = replicates[replicates['library'] == alias]
        for filename in sftp.listdir(dirname):
            if filename.endswith('fastq.gz'):
                relative_path = os.path.join(dirname, filename)
                metadata = {
                    'dataset': list(replicate_row['experiment'])[0],
                    'submitted_file_name': relative_path,
                    'replicate': list(replicate_row['uuid'])[0],
                    'lab': list(library_row['lab'])[0],
                    'award': list(library_row['award'])[0],
                }
                #display.display_pretty(metadata)
                print('<http://jumpgate.caltech.edu/wiki/SubmissionsLog/{}#{}>'.format(submission_name, dirname))
                print('  encode3:dataset "{}" ;'.format(metadata['dataset']))
                print('  encode3:replicate "{}" ;'.format(metadata['replicate']))
                print('  encode3:lab "{}" ;'.format(metadata['lab']))
                print('  encode3:award "{}" .'.format(metadata['award']))

                print('<file:///woldlab/castor/home/diane/proj/submission/{}/{}>'.format(submission_name, relative_path))
                print('  encode3:output_type "reads" ;')
                print('  encode3:file_format "fastq" .')
                files += 1

    print('generated {} records'.format(files))


In [18]:
make_manifest(spreadsheet_name, 'encode-y4q1', '/home/diane/proj/submission/encode-y4q1/', 'libs.txt')

<http://jumpgate.caltech.edu/wiki/SubmissionsLog/encode-y4q1#15158-forebrain>
  encode3:dataset "ENCSR889WQX" ;
  encode3:replicate "3412457b-c78d-4d31-b581-afe63873060d" ;
  encode3:lab "barbara-wold" ;
  encode3:award "U54HG006998" .
<file:///woldlab/castor/home/diane/proj/submission/encode-y4q1/15158-forebrain/15158_HCCNNADXX_c116_l2.fastq.gz>
  encode3:output_type "reads" ;
  encode3:file_format "fastq" .
<http://jumpgate.caltech.edu/wiki/SubmissionsLog/encode-y4q1#15158-forebrain>
  encode3:dataset "ENCSR889WQX" ;
  encode3:replicate "3412457b-c78d-4d31-b581-afe63873060d" ;
  encode3:lab "barbara-wold" ;
  encode3:award "U54HG006998" .
<file:///woldlab/castor/home/diane/proj/submission/encode-y4q1/15158-forebrain/15158_HBE3UADXX_c66_l1.fastq.gz>
  encode3:output_type "reads" ;
  encode3:file_format "fastq" .
<http://jumpgate.caltech.edu/wiki/SubmissionsLog/encode-y4q1#15158-forebrain>
  encode3:dataset "ENCSR889WQX" ;
  encode3:replicate "3412457b-c78d-4d31-b581-afe63873060d" ;
  

/usr/lib/python3/dist-packages/paramiko/client.py:580: UserWarning: Unknown ssh-rsa host key for pongo.cacr.caltech.edu: b'6833531597d6176b45c27f33e9406a12'
  (key.get_name(), hostname, hexlify(key.get_fingerprint())))
